# Model Training

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/gemstone.csv')
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df.drop(labels=['id'],axis=1)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [6]:
X

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [7]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [8]:
# Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [10]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
# Numeric pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
    ]
)

# Catagrical pipeline
cat_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]
)
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [19]:
# spilit Train and Test data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33,random_state=42)

In [20]:
# Standarscaler
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [21]:
X_train

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.887629,2.626061,-2.888129,0.400868,2.233112,2.216066,1.856561,-0.130933,1.525655,-1.314696
1,-1.057304,-0.845291,0.164716,0.922458,-0.915966,-0.908068,-0.890852,-0.130933,-0.937159,-0.648656
2,-0.002860,-0.845291,-1.500472,1.965640,-0.843987,-0.899013,-0.963153,-0.130933,-0.321455,-0.648656
3,0.981911,-0.694363,-0.667878,-0.642314,-0.637048,-0.636405,-0.673951,0.874463,-0.937159,-1.314696
4,0.030009,1.548002,-0.482857,1.444049,1.477333,1.455407,1.393839,-0.130933,1.525655,0.683424
...,...,...,...,...,...,...,...,...,...,...
129688,0.414836,-0.629679,-1.500472,1.965640,-0.547074,-0.518684,-0.645031,-1.136330,-0.937159,-0.648656
129689,0.125398,2.410449,0.442247,2.487231,1.918204,1.871959,1.928861,-1.136330,-0.321455,-0.648656
129690,0.630382,0.922727,0.904800,0.400868,0.991476,0.921135,1.046797,-0.130933,0.294248,0.017384
129691,0.897467,-1.039342,-0.667878,-0.642314,-1.212879,-1.197843,-1.252354,-1.136330,0.294248,2.015504


In [22]:
X_test

,num_pipeline__id,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.463091,-0.629679,0.257226,-0.120723,-0.601058,-0.582072,-0.572731,0.874463,-1.552862,-0.648656
1,1.232704,2.604500,-2.148045,-0.120723,2.125144,2.197955,1.957782,-1.136330,0.294248,-1.314696
2,0.000806,-1.125587,-1.222941,0.922458,-1.374832,-1.415174,-1.469255,-0.130933,-0.937159,2.015504
3,0.874684,-1.017781,-0.575368,0.922458,-1.158895,-1.161621,-1.194514,-0.130933,1.525655,2.015504
4,0.393555,0.858043,0.627268,-0.642314,0.946489,0.984523,1.003417,0.874463,0.909952,-0.648656
...,...,...,...,...,...,...,...,...,...,...
63875,-0.236325,-0.823730,-0.390347,0.922458,-0.870979,-0.862791,-0.876392,-0.130933,-0.937159,0.683424
63876,1.650884,0.599309,-0.482857,-1.163905,0.811528,0.839636,0.772056,0.874463,0.294248,0.017384
63877,0.636927,-0.500312,-0.482857,0.922458,-0.358129,-0.410018,-0.442590,-0.130933,-0.937159,0.017384
63878,0.213723,-0.133772,0.442247,0.922458,0.028757,0.060866,0.077972,-1.136330,1.525655,-1.314696


In [25]:
# Model Traning
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [26]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [49]:
models ={
    'LinearRegression':LinearRegression(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'ElasticNet':ElasticNet()
}
model_list =[]
r2_list=[]
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    y_pred =model.predict(X_test)
    mae, rmse, r2_square = evaluate_model(y_test,y_pred)
    #model name
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    r2_list.append(r2_square)
    print('='*35)
    # print(model_list)
    # print(r2_list)
print('')
print("Highest Accurcy Model is :",model_list[r2_list.index(max(r2_list))],'\n''Accuracy is:',max(r2_list))

LinearRegression
Model Training Performance
RMSE: 1014.2778153382556
MAE: 674.7348151188762
R2 score 93.63889934551443
Ridge
Model Training Performance
RMSE: 1014.2820747091463
MAE: 674.7683132311179
R2 score 93.63884591963162
Lasso
Model Training Performance
RMSE: 1014.3372093373042
MAE: 675.8980580607733
R2 score 93.63815433805705
ElasticNet
Model Training Performance
RMSE: 1534.1040071498533
MAE: 1061.324860120264
R2 score 85.44782116433484

Highest Accurcy Model is : LinearRegression 
Accuracy is: 0.9363889934551443
